In [7]:
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("/kaggle/input/xception-stanford-dog/keras/default/1/best_finetuned_model.keras")  # Change to your model file

In [8]:
import numpy as np
from tensorflow.keras.preprocessing import image

IMG_SIZE = 299  # Ensure this matches the model input size

def preprocess_image(img_path):
    """Load and preprocess an image for prediction."""
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize to [0,1] (same as training)
    return img_array

In [9]:
def predict_image(img_path, model, class_mapping):
    """Predict the breed of the given image."""
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    
    predicted_index = np.argmax(predictions)  # Get index of highest probability
    predicted_breed = None

    # Find breed name from index
    for breed, data in class_mapping.items():
        if data['index'] == predicted_index:
            predicted_breed = data['name']
            break

    return predicted_breed, predictions[0][predicted_index]  # Breed and confidence score


In [13]:
import json

# Load class mapping from the file where you stored it
with open("//kaggle/working/class_mapping.json", "r") as f:
    class_mapping = json.load(f)

# Provide an image path
img_path = "/kaggle/input/random-101/download.jfif"  # Replace with your test image path

# Run prediction
predicted_breed, confidence = predict_image(img_path, model, class_mapping)

print(f"Predicted Breed: {predicted_breed} ({confidence:.2%} confidence)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted Breed: schipperke (7.43% confidence)


In [3]:
import json
import os

DATASET_PATH = "/kaggle/input/stanford-dogs-dataset"
IMAGES_PATH = os.path.join(DATASET_PATH, "images", "Images")

def parse_breed_name(folder_name):
    """Extract breed name from folder name (e.g., 'n02085620-Chihuahua' -> 'Chihuahua')"""
    return folder_name.split('-')[1]

def generate_class_mapping():
    """Generate class mapping from dataset folders."""
    breed_folders = [f for f in os.listdir(IMAGES_PATH) if os.path.isdir(os.path.join(IMAGES_PATH, f))]
    
    class_mapping = {folder: {"name": parse_breed_name(folder), "index": idx} 
                     for idx, folder in enumerate(sorted(breed_folders))}
    
    with open("class_mapping.json", "w") as f:
        json.dump(class_mapping, f, indent=4)

    print("class_mapping.json has been generated.")

if __name__ == "__main__":
    generate_class_mapping()


class_mapping.json has been generated.
